In [ ]:
import sys
sys.path.insert(0, "/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/saga")

In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H

In [ ]:
saga = SAGA.QuickStart()

In [ ]:
specs_1 = FitsTable(saga._shared_dir + "/Paper1/saga_spectra_clean.fits.gz").read()

In [ ]:
specs_1["HOSTID"] = np.char.add("nsa", specs_1["HOST_NSAID"].astype("<7U"))
specs_1["r_mag"] = specs_1["r"] - specs_1["EXTINCTION_R"]
specs_1["Mr"] = specs_1["r_mag"] - np.log10(specs_1["HOST_DIST"])*5 - 25

In [ ]:
specs_2 = saga.object_catalog.load_clean_specs()

In [ ]:
hostlist = saga.host_catalog.load(add_coord=False, include_stats=True)

paper1_complete = saga.host_catalog.construct_host_query("paper1_complete")
paper1_observed = saga.host_catalog.construct_host_query("paper1_observed")
paper2_complete = saga.host_catalog.construct_host_query("paper2_complete")
paper2_observed = saga.host_catalog.construct_host_query("paper2_observed")
paper3_planned = QueryMaker.in1d("HOSTID", hostlist["HOSTID"][hostlist["really_need_spec"].argsort()[:100]].tolist())

In [ ]:
q = Query(C.basic_cut2, C.has_spec, "SPEC_Z > 0.001", paper2_complete, C.main_targeting_cuts)

ratio_f = (q & C.is_sat & (~C.sdss_limit)).count(specs_2) / (q & (~C.sdss_limit)).count(specs_2) 
ratio_b = (q & C.is_sat & C.sdss_limit).count(specs_2) / (q & C.sdss_limit).count(specs_2)

In [ ]:
X = []

for i, sel in enumerate([paper1_complete, paper1_observed, paper2_complete, paper2_observed, paper3_planned]):
    if i < 2:
        clean_galaxy = Query(C.is_clean, C.is_galaxy, "SPEC_Z > 0.001")
        specs = specs_1
    else:
        clean_galaxy = Query(C.is_clean2, C.is_galaxy2, "SPEC_Z > 0.001")
        specs = specs_2
    
    if i == 4:
        spec_funge = sel.filter(hostlist, "really_need_spec").sum() + (60 * sel.count(hostlist))
        nb = sel.filter(hostlist, "really_need_spec_bright").sum()
        nf = sel.filter(hostlist, "really_need_spec").sum() - nb
        sat_funge = int((nb * ratio_b + nf * ratio_f))
    else:
        spec_funge = 0
        sat_funge = 0
        
    X.append((
        sel.count(hostlist),
        Query(clean_galaxy, sel, C.has_our_specs_only).count(specs) + spec_funge,
        Query(clean_galaxy, sel).count(specs) + spec_funge,
        Query(clean_galaxy, sel, C.is_sat, C.has_our_specs_only, C.r_abs_limit).count(specs) + int(sat_funge*1.05),
        Query(clean_galaxy, sel, C.is_sat, C.r_abs_limit).count(specs) + int(sat_funge*1.05),
        Query(clean_galaxy, sel, C.is_sat, C.has_our_specs_only, ~C.r_abs_limit).count(specs),
        Query(clean_galaxy, sel, C.is_sat, ~C.r_abs_limit).count(specs),
    ))
    
X = np.array(X).T
X